In [19]:
import os

import awswrangler as wr
import fsspec
import pandas as pd
from aodndata.soop.soop_xbt_nrt import parse_bufr_file
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from elasticsearch_dsl.connections import connections


In [3]:
BUCKET = "imos-data-lab-raw"
PATH = "IMOS/SOOP/SOOP-XBT/REALTIME_BUFR"

BUCKET_OPTIMISED = "imos-data-lab-optimised"
PATH_OPTIMISED = "TEST_DATA/XBT"

In [4]:
# Use FSSpec to get a list of files in the bucket at the path
fs = fsspec.filesystem('s3')

files = ['s3://' + f for f in fs.glob(f"s3://{BUCKET}/{PATH}/*/*.csv")]
files[0]

's3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20200901134700_D5LR9.csv'

In [32]:
def process_file(file, test=False):
    print(f"Processing file {file}")
    profiles = parse_bufr_file(file)

    for profile in profiles:
        metadata = {
            "profile_geotime": profile["profile_geotime"],
            "profile_metadata": profile["profile_metadata"],
        }

        out_file = f'{metadata["profile_geotime"]["date_utc"]:%Y%m%dT%H%M}_{metadata["profile_metadata"]["XBT_uniqueid"]}'
        metadata["profile_geotime"][
            "date_utc"
        ] = f'{metadata["profile_geotime"]["date_utc"]:%Y-%m-%dT%H:%M:00}'

        p = profile["profile_data"]
        data_headers = ["depth", "temp"]
        data_headers_array = ["glob_gtspp", "glob_gtspp_depth", "glob_gtspp_temp"]

        to_zip = [list(p[var].values) for var in data_headers] + [
            list(p[var]) for var in data_headers_array
        ]
        data = list(zip(*to_zip))

        point = f'POINT ({metadata["profile_geotime"]["longitude"]} {metadata["profile_geotime"]["latitude"]})'
        coords = [metadata["profile_geotime"]["longitude"], metadata["profile_geotime"]["latitude"]]
        time = metadata["profile_geotime"]["date_utc"]
        uid = metadata["profile_metadata"]["XBT_uniqueid"]

        df = pd.DataFrame(data, columns=data_headers + data_headers_array)
        df["uid"] = uid
        df["geom"] = point
        df["datetime"] = time
        df['datetime'] = pd.to_datetime(df.datetime)
        for var in data_headers_array:
            df[var] = df[var].astype(int)

        for row in df.iterrows():
            data = row[1]
            yield {
                "_index": "soop_xbt",
                "temp": data.temp,
                "depth": data.depth,
                "glob_gtspp": data.glob_gtspp,
                "glob_gtspp_depth": data.glob_gtspp_depth,
                "glob_gtspp_temp": data.glob_gtspp_temp,
                "uid": data.uid,
                "location": coords,
                "datetime": data.datetime
            }


In [37]:
client = Elasticsearch(
    "https://foss4g-workshop.es.europe-west3.gcp.cloud.es.io",
    http_auth=("elastic", os.environ.get("ES_KEY", "")),
)

# client.indices.create(
#     "soop_xbt",
#     ignore=400,
#     body={"mappings": {"properties": {"location": {"type": "geo_point"}}}},
# )

for file in files:
    print(f"Processing file {file}")
    bulk(client, process_file(file))
    # break



Processing file s3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20200901134700_D5LR9.csv
Processing file s3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20200901134700_D5LR9.csv


/usr/local/lib/python3.9/site-packages/aodndata/soop/soop_xbt_nrt.py:190: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(csv_path, header=None, engine="python", error_bad_lines=False)


Processing file s3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20200901170100_D5LR9.csv
Processing file s3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20200901170100_D5LR9.csv


/usr/local/lib/python3.9/site-packages/aodndata/soop/soop_xbt_nrt.py:190: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(csv_path, header=None, engine="python", error_bad_lines=False)


Processing file s3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20200901210400_D5LR9.csv
Processing file s3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20200901210400_D5LR9.csv


...

Processing file s3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20201122144700_XEKXW9W.csv
Processing file s3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20201122144700_XEKXW9W.csv


/usr/local/lib/python3.9/site-packages/aodndata/soop/soop_xbt_nrt.py:190: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(csv_path, header=None, engine="python", error_bad_lines=False)


Processing file s3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20201122170400_XEKXW9W.csv
Processing file s3://imos-data-lab-raw/IMOS/SOOP/SOOP-XBT/REALTIME_BUFR/2020/IOSS01_AMMC_20201122170400_XEKXW9W.csv


/usr/local/lib/python3.9/site-packages/aodndata/soop/soop_xbt_nrt.py:190: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(csv_path, header=None, engine="python", error_bad_lines=False)


PermissionError: The provided token has expired.

In [2]:
wr.s3.read_parquet("s3://imos-data-lab-optimised/TEST_DATA/XBT-WRANGLER/").query()


wr.s3.read_partu

ClientError: An error occurred (ExpiredToken) when calling the ListObjectsV2 operation: The provided token has expired.

In [3]:
import awswrangler as wr

table = wr.athena.read_sql_query("select * from soop_xbt_nrt limit 5", "profile")


In [5]:
type(table)

pandas.core.frame.DataFrame

In [6]:
import dask.dataframe as dd


/usr/local/lib/python3.9/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.9/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.9/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [9]:
%%time

import dask.dataframe as dd
df = dd.read_parquet('s3://imos-data-lab-optimised/parquet/xbt/', columns=['temp'], engine='pyarrow')
df.temp.mean().compute()

CPU times: user 1.84 s, sys: 269 ms, total: 2.11 s
Wall time: 2.17 s


8.40091803179103